# Простой пример реализации внимание (self-attention) и эмбеддинг для вещественных чисел.

In [2]:
import numpy as np

# Пример: последовательность длиной 5, по 1 признаку (просто числа)
sequence = np.array([[0.1], [0.2], [0.3], [2.0], [0.4]])  # (5, 1)

# Параметры
seq_len, input_dim = sequence.shape
d_model = 4  # размерность эмбеддинга
print(f"sequence:\n {sequence}")
print(f"seq_len: {seq_len}")
print(f"input_dim: {input_dim}")


sequence:
 [[0.1]
 [0.2]
 [0.3]
 [2. ]
 [0.4]]
seq_len: 5
input_dim: 1


In [3]:
# 🔹 Шаг 1: Эмбеддинг (линейная проекция чисел)

# Линейная проекция входных чисел в d_model
W_embed = np.random.randn(input_dim, d_model)
print(f"W_embed:\n {W_embed}")
embedded = sequence @ W_embed  # матричное перемножение -> (5, d_model) 
# линейная проекцию входной последовательности чисел в векторное пространство
# большей размерности (аналог слоя эмбеддинга в трансформерах).
# т.е. переводим скалярные значения в векторное пространство
print(f"embedded:\n {embedded}")

W_embed:
 [[-0.40864155  1.24500152 -0.35641933  0.5953186 ]]
embedded:
 [[-0.04086415  0.12450015 -0.03564193  0.05953186]
 [-0.08172831  0.2490003  -0.07128387  0.11906372]
 [-0.12259246  0.37350045 -0.1069258   0.17859558]
 [-0.8172831   2.49000303 -0.71283867  1.1906372 ]
 [-0.16345662  0.49800061 -0.14256773  0.23812744]]


In [4]:
# 🔹 Шаг 2: Генерация Q, K, V

# Матрицы для attention
W_Q = np.random.randn(d_model, d_model)
W_K = np.random.randn(d_model, d_model)
W_V = np.random.randn(d_model, d_model)

Q = embedded @ W_Q
K = embedded @ W_K
V = embedded @ W_V

In [ ]:
# 🔹 Шаг 3: Attention механизм

# Attention = softmax(QK^T / sqrt(d)) V
def softmax(x):
    e_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    # Что делает x - np.max(...):
    # Из каждого элемента вектора вычитается 
    # максимум по этому вектору. Зачем:
    # Это трюк для численной стабильности, 
    # чтобы избежать переполнения при экспоненте.
    
    # Что делает np.exp(...):
    # Применяет экспоненту к каждому элементу массива.
    # Это делает большие значения ещё больше, 
    # а маленькие — ближе к нулю. 
    # Тем самым подчёркивается разница 
    # между важными и неважными элементами.
    
    return e_x / e_x.sum(axis=-1, keepdims=True)
    # Что делает e_x.sum(...):
    # Считает сумму всех экспоненцированных значений.
    # Что делает e_x / sum:
    # Нормализует значения: 
    # делит каждое значение на общую сумму 
    # → получается вероятность.

    # Что делает функция в целом
    # Вход:  x = [2.0, 1.0, 0.1]
    # Выход: softmax(x) = [0.659, 0.242, 0.098]
    
    # Преобразует произвольные числа в "веса внимания" (или вероятности),
    # Выделяет более важные элементы (самые большие значения),
    # Все значения положительны и в сумме дают 1.

In [ ]:
scores = Q @ K.T / np.sqrt(d_model)  # (5, 5)
# ключевой шаг механизма внимания (self-attention).

# 🔹 1. Q @ K.T — сравнение между запросами и ключами
# Q (Queries) — запросы: "чему мне нужно уделить внимание?"
# K (Keys) — ключи: "что у нас есть в памяти?"
# Операция Q @ K.T — это матричное умножение запросов 
# на транспонированные ключи, что даёт оценку «сходства» 
# или «важности» каждого элемента относительно всех остальных.
# Q: (seq_len, d_model)
# K.T: (d_model, seq_len)
# → scores: (seq_len, seq_len)
# Результат: матрица scores, где каждый элемент scores[i, j] 
# показывает, насколько i-й элемент внимания "смотрит" на j-й.

# 🔹 2. Деление на sqrt(d_model) — нормализация
# Это масштабирование, предложенное в оригинальной статье 
# Attention Is All You Need (2017)- https://arxiv.org/abs/1706.03762.
# Зачем?
# Если размерность d_model большая, скалярные произведения 
# Q @ K.T становятся слишком большими. Это приводит к:
# резкой контрастности softmax (всё внимание на 1 элемент),
# плохой обучаемости (градиенты затухают или взрываются).
# 📏 Решение: делим на корень из размерности, 
# чтобы значения остались в «нормальном» диапазоне.

$$
\text{scores}_{i,j} = \frac{Q_i \cdot K_j}{\sqrt{d_\text{model}}}
$$

In [ ]:
# Далее мы преобразуем оценки в вероятности
weights = softmax(scores)           # (5, 5)

In [ ]:
attention_output = weights @ V      # (5, d_model)
# Здесь значения V (value vectors, "содержимое") взвешивается 
# по весам внимания.
# То есть каждый выходной вектор (строка context[i]) — 
# это суммарное значение, куда "впиталась" информация 
# от всех других векторов, но с разной важностью.

# Как происходит процессе обучения модели.

С использованием градиентного спуска и обратного распространения ошибки (backpropagation). 

__1. Случайная инициализация весов.__  
__2. Обучение через градиентный спуск:__  
Во время обучения происходит следующий процесс:  
_2.1. Прямой проход (forward pass):_  
На первом шаге модель вычисляет все промежуточные значения: 
$Q, 𝐾, V$, затем вычисляются оценки важности с помощью 
$ 𝑄@𝐾^{𝑇} $, далее применяются softmax для получения весов, и, наконец, вычисляется контекст.  
_2.2. Расчёт ошибки:_  
После этого вычисляется ошибка (или потеря, например, с использованием функции потерь, такой как кросс-энтропия для классификации или среднеквадратичная ошибка для регрессии) между предсказанным результатом и фактическим значением.
_2.3. Обратное распространение (backpropagation):_  
Ошибка используется для вычисления градиентов (производных) функции потерь по всем параметрам модели — включая - $ 𝑊_Q, W_K, W_V $. Градиенты показывают, насколько сильно нужно изменить каждый из весов для минимизации ошибки. Например, для $ 𝑊_{q}, W_{k}, W_{v}. $ градиент будет выглядеть так:  
$$ 
\frac{∂loss}{∂W_Q}
$$​
_2.4. Обновление весов:_  
Градиенты используются для обновления весов с использованием метода градиентного спуска. Обновление веса происходит по формуле:
$$
𝑊 = 𝑊 − 𝜂\cdot{\frac{∂loss}{∂W_Q}}
$$

Где:
𝜂 — скорость обучения (learning rate) — шаг, на который мы обновляем веса.
Например, для обновления $ W_Q $:  
$$
W_Q = W_Q - \text{learning\_rate} \times \text{grad\_W\_Q}
$$  

Этот процесс повторяется множество раз (на каждом шаге обучения), пока веса не научатся корректно отображать входные данные в полезные эмбеддинги.

__3. Как меняются веса в процессе обучения?__  
На каждом шаге градиентного спуска веса $W_Q, W_K, W_V$ корректируются таким образом, чтобы минимизировать ошибку модели.  
Веса будут изменяться в зависимости от того, какие элементы последовательности имеют большее или меньшее влияние на результат. 
Например:  
Если запросы $𝑄$ и ключи $𝐾$ сильно схожи, модель будет учить веса так, чтобы вычисляемые "оценки важности" были высокими для этих пар.  
Если значения $V$ менее важны для конкретного контекста, модель может уменьшить вес, который присваивает этим элементам внимания.

__🧩 Пример в контексте обучения:__  
_Начальная инициализация:_  
```
W_Q = np.random.randn(d_model, d_model)  # случайные значения
W_K = np.random.randn(d_model, d_model)
W_V = np.random.randn(d_model, d_model)
```
_Прямой проход:_  
```
Q = embedded @ W_Q  # Q = входные данные * W_Q
K = embedded @ W_K  # K = входные данные * W_K
V = embedded @ W_V  # V = входные данные * W_V
```

```
scores = Q @ K.T / np.sqrt(d_model)  # Оценки важности
weights = softmax(scores)  # Нормализуем оценки в вероятности
context = weights @ V  # Взвешиваем значения для получения 
контекста
```
_Потери и градиентный спуск:_  
После того, как контекстный вектор context вычислен, модель вычисляет ошибку на основе предсказания и фактического результата (например, для классификации или регрессии). Затем вычисляются градиенты и веса обновляются.